In [24]:
import numpy as np
import string

In [39]:
api_key = "PASTE YOUR KEY HERE"

In [26]:
characters = list(string.ascii_lowercase)  # ['a', 'b', 'c', ..., 'z']
char_to_idx = {char: idx for idx, char in enumerate(characters)}

dimension = 50
char_embeddings = np.random.uniform(-0.1, 0.1, (len(characters), dimension))

In [27]:
print(char_embeddings[0])

[ 0.01534347 -0.06973809 -0.06431585  0.00859809 -0.00527008  0.00765575
 -0.03652599  0.01627339 -0.07430283  0.05011515 -0.03723685 -0.09766947
  0.08168058 -0.0250968  -0.01988503 -0.0710732  -0.08676613 -0.08336481
  0.01451135 -0.06538733 -0.0538215  -0.01201928 -0.083459   -0.09182281
 -0.00175482  0.05899312 -0.07208533 -0.04883641  0.04389491  0.07600584
 -0.01574123  0.02580365  0.06592005  0.01955442  0.05556406  0.01457036
  0.04819666  0.06890085 -0.00935807 -0.03672484 -0.0666193  -0.03732303
 -0.00446185 -0.02117909 -0.07774656  0.0054326   0.07255876 -0.07992649
 -0.04102262 -0.08897226]


In [31]:
def get_word_embedding(word):
    embeddings = []
    for char in word:
        idx = char_to_idx[char]
        # Convert the NumPy array to a Python list
        embeddings.append(char_embeddings[idx].tolist())
    return embeddings

In [7]:
#words = []

#with open("words.txt", "r") as file:
 # for line in file:
  #  words.append(line.strip())

In [8]:
#word_by_char_embeddings = {word : get_word_embedding(word) for word in words}

#word_by_char_embeddings_list = {
 #   word: [embedding.tolist() for embedding in embeddings]
  #  for word, embeddings in word_by_char_embeddings.items()
#}

#key, value = next(iter(word_by_char_embeddings_list.items()))
#print(key, value)

#Cosine Similarity

In [29]:
def cosine_similarity(A, B):
    return np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B))

#Main Body of the game

In [8]:
mastery_levels = {0 : "beginner", 1 : "intermediate", 2 : "advanced", 3 : "Expert", 4 : "Master"}

curr_level = 0;

In [21]:
import requests
import random

def generate_word(mastery_level):
    if mastery_level == "beginner":
        prompt = f"Please provide a single English noun that is exactly 5 letters long, suitable for a word guessing game. Random seed: {random.randint(1, 1000)}."
    elif mastery_level == "intermediate":
        prompt = f"Please provide a single, commonly used English noun that is in the range of 6 to 10 letters long, suitable for a word guessing game. Random seed: {random.randint(1, 1000)}."
    elif mastery_level == "advanced":
        prompt = f"Please provide a single, uncommon English noun that is in the range of 6 to 10 letters long, suitable for a word guessing game. Random seed: {random.randint(1, 1000)}."
    elif mastery_level == "Expert":
        prompt = f"Please provide a single English noun that is in the range of 6 to 10 letters long and that can be considered a specialized term, suitable for a word guessing game. Random seed: {random.randint(1, 1000)}."
    else:
        prompt = f"Please provide a single English noun that is in the range of 6 to 10 letters long and can be considered challenging, suitable for a word guessing game. Random seed: {random.randint(1, 1000)}."

    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    data = {
        "model": "gpt-4",
        "messages": [
            {"role": "system", "content": "You are an assistant that generates words for a word guessing game."},
            {"role": "user", "content": prompt}
        ],
        "max_tokens": 10,
        "temperature": 1.5,
        "top_p": 0.9
    }

    try:
        response = requests.post(url, headers=headers, json=data)

        if response.status_code == 200:
            result = response.json()
            word = result['choices'][0]['message']['content'].strip().lower()

            if word.isalpha():
                return word
            else:
                return generate_word(mastery_level)
        else:
            print(f"Error: {response.status_code}, {response.json()}")
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


In [38]:
import random

welcome_message = """
🎉 Welcome to the Ultimate Hacking Game! 🎉

Get ready to test your hacking skills and have some fun!

🔹Can you guess the password to the terminal before you run out of attempts?
🔹On each attempt, you will be shown the similarity between your guess and the word. Use it to determine your next guess!
🔹If you guessed a word, you will gain points and increase your hacking level!

Best of luck, and may the words be ever in your favor! 💡✨
"""
print(welcome_message)

curr_attempts = 0
curr_level = 0

while True:

    sim_letters = 0
    secret_word = generate_word(mastery_levels[curr_level // 10])
    print(f"Current mastery level: {mastery_levels[curr_level // 10]}")
    print(f"Secret word length: {len(secret_word)}")

    guess = input("Enter your guess or type 'Exit' to quit: ").strip().lower()

    if guess == "exit":
        print("Thank you for playing!")
        break

    if len(guess) != len(secret_word):
        print(f"Your guess must be {len(secret_word)} letters long. Try again!")
        continue

    curr_attempts += 1

    guessEmbedded = get_word_embedding(guess)
    wordEmbedded = get_word_embedding(secret_word)

    for i in range(len(secret_word)):
        if i < len(guessEmbedded) and cosine_similarity(guessEmbedded[i], wordEmbedded[i]) == 1:
            sim_letters += 1

    print(f"Similarity: {sim_letters}/{len(secret_word)} letters match exactly.")

    if sim_letters == len(secret_word):
        print("🎉 You guessed the word! 🎉")
        curr_level += 1
        curr_attempts = 0
        print(f"Level up! You're now at {mastery_levels[curr_level // 10]}.")
        continue

    print(f"Attempts: {curr_attempts}. Keep trying!\n")



🎉 Welcome to the Ultimate Hacking Game! 🎉

Get ready to test your hacking skills and have some fun!

🔹Can you guess the password to the terminal before you run out of attempts?
🔹On each attempt, you will be shown the similarity between your guess and the word. Use it to determine your next guess!
🔹If you guessed a word, you will gain points and increase your hacking level!

Best of luck, and may the words be ever in your favor! 💡✨

Current mastery level: beginner
Secret word length: 5
Enter your guess or type 'Exit' to quit: Quite
Similarity: 1/5 letters match exactly.
Attempts: 1. Keep trying!

Current mastery level: beginner
Secret word length: 5
Enter your guess or type 'Exit' to quit: Piles
Similarity: 0/5 letters match exactly.
Attempts: 2. Keep trying!

Current mastery level: beginner
Secret word length: 5
Enter your guess or type 'Exit' to quit: Dirty
Similarity: 0/5 letters match exactly.
Attempts: 3. Keep trying!

Current mastery level: beginner
Secret word length: 5
Enter yo